In [1]:
import sys
import os

# Insert the parent directory of "app" into sys.path
# so that Python recognizes "app" as an importable package.
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(parent_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv()  # This looks for .env in the current working directory

True

In [3]:
import time

import pandas as pd
import seaborn as sns
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    context_recall,
    faithfulness
)

from langchain_core.messages import HumanMessage, ToolMessage, AIMessage, ToolMessage


In [4]:
# Benchmark data
test_df = pd.read_csv('benchmark.csv', delimiter=";")
test_df.head()

,question,ground_truth
0,What are some contracts that ended after 1990-...,There are a total of 377 contracts that ended ...
1,What are some contracts that ended after 1990-...,"There are 48 contracts classified as ""Strategi..."
2,"What are some contracts with HC2 Holdings, Inc.?","There is one contract involving HC2 Holdings, ..."
3,Which year was the most contracts signed?,The year with the most contracts signed was 20...
4,Which company has the most active contracts?,The company with the most active contracts is ...


In [5]:
from backend.src.agent import get_agent

In [6]:
agent = get_agent()

In [7]:
input_messages = [HumanMessage(content="Which day is today?")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which day is today?
================================== Ai Message ==================================

Today is 2025-03-19.


In [8]:
# History
messages

{'messages': [HumanMessage(content='Which day is today?', additional_kwargs={}, response_metadata={}, id='4b8afdd3-95a1-4594-9aed-3c87ca1732ba'),
  AIMessage(content='Today is 2025-03-19.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-6ac03913-65df-4573-9f41-29fad6cb93a1-0', usage_metadata={'input_tokens': 683, 'output_tokens': 15, 'total_tokens': 698, 'input_token_details': {'cache_read': 0}})]}

In [9]:
# Chat with history
input_messages = messages['messages'] + [HumanMessage(content="List all contracts where a party from Japan is involved.")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which day is today?
================================== Ai Message ==================================

Today is 2025-03-19.
================================ Human Message =================================

List all contracts where a party from Japan is involved.
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (0d68bb74-0823-4f1a-84ad-84f091999dd1)
 Call ID: 0d68bb74-0823-4f1a-84ad-84f091999dd1
  Args:
    country: JP
================================= Tool Message =================================
Name: ContractSearch

[{"output": {"total_count_of_contracts": 0, "example_values": []}}]
================================== Ai Message ==================================

There are no contracts involving a party from Japan.


In [10]:
def get_answer(input: str) -> str:
    input_messages = [HumanMessage(content=input)]
    messages = agent.invoke({"messages": input_messages})
    tools = [m.additional_kwargs.get('function_call') for m in messages["messages"] if isinstance(m, AIMessage) and m.additional_kwargs.get('function_call')]
    context = [m.content for m in messages["messages"] if isinstance(m, ToolMessage)]
    answer = messages["messages"][-1].content
    return answer, tools, context

In [22]:
from tqdm import tqdm

# Get workflows generated answers
results = []
latencies = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing questions"):
    question = row['question']
    start = time.time()
    try:
        data = get_answer(question)
    except:
        data = {"answer": "timeout", "question": question}
    end = time.time()
    latencies.append(end - start)
    results.append(data)

test_df["latencies"] = latencies
test_df["response"] = [el[0] for el in results]
test_df["tools"] = [el[1] for el in results]
test_df["retrieved_contexts"] = [[str(item) for item in el[2]] for el in results]

Processing questions: 100%|██████████| 44/44 [01:08<00:00,  1.57s/it]


In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Adjust width to prevent wrapping
pd.set_option('display.max_colwidth', None)  # Prevents truncation of long column values

In [23]:
test_df[['response','question', 'tools']].head()

response  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        Here are some contracts with end dates after 1990-01-01. The tool found 323 contracts. I am including a few examples in this response:\n*   **Strategic Alliance**: This contract is between PHL VARIABLE INSURANCE COMPANY, PHOENIX LIFE INSURANCE COMPANY, PHOENIX EQUITY PLANNING CORPOR

In [24]:
dataset = Dataset.from_pandas(test_df)

result = evaluate(
    dataset,
    metrics=[
        answer_correctness,
        context_recall,
        faithfulness
    ],
)

Evaluating:   0%|          | 0/132 [00:00<?, ?it/s]

Exception raised in Job[15]: KeyError('reference')
Exception raised in Job[16]: KeyError('reference')
Exception raised in Job[19]: KeyError('reference')
Exception raised in Job[18]: KeyError('reference')
Exception raised in Job[22]: KeyError('reference')
Exception raised in Job[21]: KeyError('reference')
Exception raised in Job[25]: KeyError('reference')
Exception raised in Job[24]: KeyError('reference')
Exception raised in Job[28]: KeyError('reference')
Exception raised in Job[31]: KeyError('reference')
Exception raised in Job[27]: KeyError('reference')
Exception raised in Job[34]: KeyError('reference')
Exception raised in Job[37]: KeyError('reference')
Exception raised in Job[30]: KeyError('reference')
Exception raised in Job[40]: KeyError('reference')
Exception raised in Job[36]: KeyError('reference')
Exception raised in Job[39]: KeyError('reference')
Exception raised in Job[43]: KeyError('reference')
Exception raised in Job[46]: KeyError('reference')
Exception raised in Job[48]: Ke

In [29]:
result.to_pandas()[['user_input','answer_correctness', 'context_recall', 'faithfulness', 'response']]

user_input  \
0                                      What are some contracts that ended after 1990-01-01?   
1   What are some contracts that ended after 1990-01-01 that fall under strategic alliance?   
2                                          What are some contracts with HC2 Holdings, Inc.?   
3                                                 Which year was the most contracts signed?   
4                                              Which company has the most active contracts?   
5                                     Which Hosting contracts have the highest total value?   
6                                     How many contracts does Microgenics Corporation have?   
7                                                    What contracts are governed by US law?   
8                                    Which party has the most Strategic Alliance contracts?   
9                       What is the total monetary value of all contracts with WebMD, Inc.?   
10                                       How many parties are located in the United States?   
11                             Which contracts have a total amount greater than $1,000,000?   
12                             What are the contract types used by Snap Technologies, Inc.?   
13                             Which countries have the most parties involved in contracts?   
14                                     What is the average contract value for IP contracts?   
15             List all Joint Venture contracts involving parties from different countries.   
16                                              Which contract has the longest term period?   
17                         Which parties have both License Agreement and Service contracts?   
18                              What is the distribution of contract types in our database?   
19                              List all contracts signed between January and October 2000.   
20                                  Which party has contracts with the highest total value?   
21                                        How many contracts have Achaogen Inc. as a party?   
22                         List all Supply contracts with a total amount more than $50,000.   
23                                          Which contracts have the most parties involved?   
24                                   What is the average duration of Distributor contracts?   
25                                        How many Franchise contracts were signed in 2020?   
26                             Which contracts involve parties from both the US and Canada?   
27                       Find the most common contract type for parties located in Germany.   
28                                   What percentage of contracts are Marketing agreements?   
29                         Which month saw the highest number of contract signings in 2000?   
30                                      List all contracts with a term longer than 5 years.   
31                      What is the most common governing law for Transportation contracts?   
32                                              Which Sponsorship contracts expire in 2015?   
33                                 List all contracts where a party from Japan is involved.   
34                                    Which Hosting contracts have the highest total value?   
35       How many contracts started after January 1, 2020 and end before December 31, 2023?   
36                 Which contracts with Microgenics Corporation ended in the past 6 months?   
37            How many contracts have an effective date before 2015 but are still in force?   
38                   Find all License Agreements that started in 2015 and are still active.   
39                              How many contracts signed in 2005 have an end date in 2006?   
40    Find all contracts that had a duration of more than 24 months and have already ended.   
41   List all IP contracts with effective dates between June 1, 2018 and December 31, 2020.